In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy import stats
import numpy as np
%matplotlib inline

In [2]:
대주주외국인지분변화 = pd.read_csv('대주주외국인지분변화.csv',encoding='utf-8-sig',index_col=0)
산업분류_G임금 = pd.read_csv('산업분류_G임금.csv',encoding='utf-8-sig',index_col=0)
토빈Q = pd.read_csv('토빈Q.csv',encoding='utf-8-sig',index_col=0)
현금흐름 = pd.read_csv('현금흐름.csv',encoding='utf-8-sig',index_col=0)
RDS = pd.read_csv('RDS.csv',encoding='utf-8-sig',index_col=0)
target = pd.read_csv('target2.csv',encoding='utf-8-sig',index_col=0)
ww = pd.read_csv('ww지수.csv',encoding='utf-8-sig',index_col=0)

In [3]:
 #회계년도컬럼, 년도만 뽑아서 int화
def change_year(x):
    return int(x[:4])
대주주외국인지분변화["회계년도"] = 대주주외국인지분변화["회계년도"].map(change_year) # 년도만 표시

In [4]:
df = pd.merge(대주주외국인지분변화,산업분류_G임금, on=['회사명','거래소코드','회계년도'],how='inner')
df = pd.merge(df,토빈Q, on=['회사명','거래소코드','회계년도'],how='inner')
df = pd.merge(df,현금흐름, on=['회사명','거래소코드','회계년도'],how='inner')
df = pd.merge(df,RDS, on=['회사명','거래소코드','회계년도'],how='inner')
df = pd.merge(df,target, on=['회사명','거래소코드','회계년도'],how='inner')
df = pd.merge(df,ww,on=['회사명','거래소코드','회계년도'],how='inner')

In [5]:
df.shape

(14547, 47)

In [6]:
df.columns

Index(['회사명', '거래소코드', '회계년도', '대주주지분변화분', '외국인지분분변화', '상장일', '자산',
       '당기순이익(손실)', '미처분이익잉여금(결손금)', '총자본증가율', '비유동자산증가율', '유동자산증가율',
       '자기자본증가율', '매출액증가율', '정상영업이익증가율', '순이익증가율', '총포괄이익증가율', '매출액순이익률',
       '총자본순이익률', '자기자본순이익률', '매출원가대매출액비율', '광고선전비대매출액비율', '유보율', '비유동비율',
       '유동비율', '당좌비율', '현금비율', '부채비율', '유동부채비율', '차입금비율', '이자보상배율(이자비용)',
       '유보액대비율', '유보액대납입자본배율', '투자집중도', '통계청 한국표준산업분류 10차(대분류)', '직급명',
       '1인평균지급액(임원)', '1인년간평균급여(직원)', '토빈Q', '영업활동으로인한현금흐름(간접법)(천원)',
       '투자활동으로인한현금흐름(천원)', '재무활동으로인한현금흐름(천원)', '연구비', '경상개발비', '매출액(영업수익)',
       '조건2', 'WW지수'],
      dtype='object')

In [7]:
# RDS = (경상개발비 + 연구비 )/매출액
df['RDS'] = (df['경상개발비'] + df['연구비']) / df['매출액(영업수익)']
df.drop(['경상개발비','연구비','매출액(영업수익)'],axis=1,inplace=True)

In [8]:
## 기업주기 
for i in range(df.shape[0]):
    if (df.loc[i,'영업활동으로인한현금흐름(간접법)(천원)']<0) and (df.loc[i,'투자활동으로인한현금흐름(천원)']<0) and (df.loc[i,'재무활동으로인한현금흐름(천원)']>0):
        df.loc[i,'수명주기'] = '도입기'
    elif (df.loc[i,'영업활동으로인한현금흐름(간접법)(천원)']>0) and (df.loc[i,'투자활동으로인한현금흐름(천원)']<0) and (df.loc[i,'재무활동으로인한현금흐름(천원)']>0):
        df.loc[i,'수명주기'] = '성장기'
    elif (df.loc[i,'영업활동으로인한현금흐름(간접법)(천원)']>0) and (df.loc[i,'투자활동으로인한현금흐름(천원)']<0) and (df.loc[i,'재무활동으로인한현금흐름(천원)']<0):
        df.loc[i,'수명주기'] = '성숙기'
    else:
        df.loc[i,'수명주기'] = '쇠퇴기'

df.drop(['영업활동으로인한현금흐름(간접법)(천원)','투자활동으로인한현금흐름(천원)','재무활동으로인한현금흐름(천원)'],axis=1,inplace=True)

In [9]:
# B열의 값이 0 미만인 행 삭제
df = df[df['미처분이익잉여금(결손금)'] >= 0]

In [10]:
df.dropna(inplace=True)
# 해당 년도 매출액 및 영업이익의 연결범위 변동으로 인한 결측치 삭제

In [11]:
import pandas as pd

df['대주주지분변화분1'] = df['대주주지분변화분'].apply(lambda x: 1 if x <= 0 else 0)

# 결과 출력
print(df)

            회사명  거래소코드  회계년도  대주주지분변화분  외국인지분분변화         상장일            자산  \
5      (주)DB하이텍    990  2016    -25.96      3.41  1975/12/12  9.867886e+08   
6      (주)DB하이텍    990  2017     25.96      6.69  1975/12/12  9.939507e+08   
7      (주)DB하이텍    990  2018      0.00      8.16  1975/12/12  1.066534e+09   
8      (주)DB하이텍    990  2019     -0.14     11.29  1975/12/12  1.088890e+09   
9      (주)DB하이텍    990  2020      0.00    -11.99  1975/12/12  1.178581e+09   
...         ...    ...   ...       ...       ...         ...           ...   
14537   흥아해운(주)   3280  2011      1.36     40.11  1976/06/29  5.309435e+08   
14538   흥아해운(주)   3280  2012     21.09      0.92  1976/06/29  5.335714e+08   
14539   흥아해운(주)   3280  2013      9.21     -2.88  1976/06/29  5.710601e+08   
14540   흥아해운(주)   3280  2014    -28.41      1.32  1976/06/29  6.492400e+08   
14541   흥아해운(주)   3280  2015     -3.08     -6.12  1976/06/29  7.916408e+08   

         당기순이익(손실)  미처분이익잉여금(결손금)  총자본증가율  ...  통계청 한국표준산업분류 10

In [17]:
df[['대주주지분변화분1']].value_counts()

대주주지분변화분1
1            7080
0            6093
dtype: int64

In [130]:
df.to_csv('최종.csv',encoding='UTF-8-sig')